In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from keras.datasets import fashion_mnist
import cv2
import os

TRAINDIR = "ISICArchive/train_set"

TESTDIR = "ISICArchive/test_set"

CATEGORIES = ["melanoma","non_melanoma"]

IMG_SIZE = 50

Using TensorFlow backend.


In [2]:
train_x = []
train_y = []

test_x = []
test_y = []

In [3]:
def create_train_data():
    for category in CATEGORIES :
        path = os.path.join(TRAINDIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                train_x.append(new_array)
                train_y.append(class_num)
            except Exception as e:
                pass
            
            
def create_test_data():
    for category in CATEGORIES :
        path = os.path.join(TESTDIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                test_x.append(new_array)
                test_y.append(class_num)
            except Exception as e:
                pass

In [4]:
create_train_data()

In [5]:
create_test_data()

In [6]:
train_x = np.array(train_x).reshape(-1,IMG_SIZE,IMG_SIZE,1)
train_y = np.array(train_y)

test_x = np.array(test_x).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = np.array(test_y)

In [7]:
print(train_x.shape)
print(train_y.shape)

(2800, 50, 50, 1)
(2800,)


In [8]:
print(type(train_x))

<class 'numpy.ndarray'>


In [9]:
train_x = train_x.astype('float32') / 255.
test_x = test_x.astype('float32') / 255.

In [10]:
train_x = np.reshape(train_x, (len(train_x), 50, 50, 1))
test_x = np.reshape(test_x, (len(test_x), 50, 50, 1))

In [11]:
train_y = to_categorical( train_y )
test_y = to_categorical( test_y )

In [12]:
train_y.shape

(2800, 2)

In [65]:
train_y[0]

array([1., 0.], dtype=float32)

In [13]:
# Feature Extraction Layer
inputs = Input(shape=(50, 50, 1))
conv_layer = ZeroPadding2D(padding=(2,2))(inputs)
conv_layer = Conv2D(16, (5, 5), strides=(1,1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1,1), activation='relu')(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1,1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer)

In [14]:
# Flatten feature map to Vector with 576 element.
flatten = Flatten()(conv_layer)

In [15]:
# Fully Connected Layer
fc_layer = Dense(256, activation='relu')(flatten)
fc_layer = Dense(64, activation='relu')(fc_layer)
outputs = Dense(2, activation='softmax')(fc_layer)

In [16]:
model = Model(inputs=inputs, outputs=outputs)

In [17]:
# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Print Model Summary
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 1)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 54, 54, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0   

In [19]:
# Use TensorBoard
callbacks = TensorBoard(log_dir='./Graph')

In [20]:
# Train for 100 Epochs and use TensorBoard Callback
model.fit(train_x, train_y, batch_size=256, epochs=100, verbose=1, validation_data=(test_x, test_y), callbacks=[callbacks])

Train on 2800 samples, validate on 1200 samples
Epoch 1/100
2800/2800 [==============================] - 8s 3ms/step - loss: 0.6852 - accuracy: 0.5939 - val_loss: 0.6798 - val_accuracy: 0.6417
Epoch 2/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.6576 - accuracy: 0.7386 - val_loss: 0.6505 - val_accuracy: 0.6958
Epoch 3/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.6016 - accuracy: 0.7943 - val_loss: 0.5878 - val_accuracy: 0.7975
Epoch 4/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.5033 - accuracy: 0.8361 - val_loss: 0.5045 - val_accuracy: 0.8167
Epoch 5/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.3958 - accuracy: 0.8675 - val_loss: 0.4634 - val_accuracy: 0.8075
Epoch 6/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.3155 - accuracy: 0.8804 - val_loss: 0.3935 - val_accuracy: 0.8475
Epoch 7/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.272

Epoch 57/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.1036 - accuracy: 0.9639 - val_loss: 0.2142 - val_accuracy: 0.9258
Epoch 58/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.1021 - accuracy: 0.9646 - val_loss: 0.2767 - val_accuracy: 0.9100
Epoch 59/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.1044 - accuracy: 0.9607 - val_loss: 0.2246 - val_accuracy: 0.9283
Epoch 60/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.0968 - accuracy: 0.9675 - val_loss: 0.2467 - val_accuracy: 0.9192
Epoch 61/100
2800/2800 [==============================] - 6s 2ms/step - loss: 0.0925 - accuracy: 0.9700 - val_loss: 0.2339 - val_accuracy: 0.9267
Epoch 62/100
2800/2800 [==============================] - 7s 2ms/step - loss: 0.0936 - accuracy: 0.9675 - val_loss: 0.1987 - val_accuracy: 0.9358
Epoch 63/100
2800/2800 [==============================] - 7s 2ms/step - loss: 0.0978 - accuracy: 0.9639 - val_loss: 0.4093 -

In [23]:
# Save Weights
model.save_weights('weights.h5')

In [100]:
first_sample_array = cv2.imread('ISIC_0000001.jpg', cv2.IMREAD_GRAYSCALE)
first_resized_sample = cv2.resize(first_sample_array, (IMG_SIZE, IMG_SIZE))

In [101]:
first_resized_sample = first_resized_sample.astype('float32') / 255.
first_resized_sample = np.array(first_resized_sample).reshape(-1,IMG_SIZE,IMG_SIZE,1)
first_predict_sample = model.predict(first_resized_sample)

In [102]:
first_predict_sample[0][1]

0.05011035

In [103]:
second_sample_array = cv2.imread('ISIC_0000000.jpg', cv2.IMREAD_GRAYSCALE)
second_resized_sample = cv2.resize(second_sample_array, (IMG_SIZE, IMG_SIZE))

In [104]:
second_resized_sample = second_resized_sample.astype('float32') / 255.
second_resized_sample = np.array(second_resized_sample).reshape(-1,IMG_SIZE,IMG_SIZE,1)
second_predict_sample = model.predict(second_resized_sample)

In [105]:
second_predict_sample[0][1]

1.02605595e-08

In [106]:
prediction = first_predict_sample
if(prediction[0][0]<0.01):
    print("non_melanoma")
else:
    print("melanoma")

melanoma
